In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import geocoder
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import json

%matplotlib inline

# Part I - Toronto_Neighborhoods

### To scrape the data of the neighborhoods in Toronto from Wikipedia into dataframe type

In [2]:
column_name = ['PostalCode', 'Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns = column_name)

In [3]:
url_str = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = urlopen(url_str)
soup = BeautifulSoup(url, 'html.parser')

In [4]:
table = soup.find('table', attrs = {'class': 'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [c.text.strip() for c in cols]
    
    if len(cols) > 1:
        neighborhoods = neighborhoods.append({'PostalCode': cols[0], 'Borough' : cols[1], 'Neighborhood': cols[2]}, ignore_index= True)


In [5]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore the rows with a borough that is "not assigned"

In [6]:
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned']

In [7]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### merge the values of "Neighborhood" based on the "PostalCode" and "Borough"

In [8]:
neighborhoods = neighborhoods.groupby(['PostalCode', 'Borough']).agg({'Neighborhood' : ','.join}).reset_index()

### If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [9]:
neighborhoods['Neighborhood'] = np.where(neighborhoods['Neighborhood'] == 'Not assigned', neighborhoods['Borough'], neighborhoods['Neighborhood'])

In [10]:
neighborhoods.shape

(103, 3)

# Part II - Data Merge

### input latitude/longitude data and merge them

In [11]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [12]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df = neighborhoods.merge(df_geo, how = 'left', left_on= 'PostalCode', right_on = 'Postal Code')

In [14]:
#remove irrelevant columns
df.drop( 'Postal Code', axis= 1, inplace= True)

In [15]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Create a map of Toronto to visualize neighborhoods

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Part III - Clustering Analysis

### Define Foursquare Crentials and Version

### Create a function to loop the process of getting the top 100 venues that are in each neighborhood  within a radius of 500 meters

In [19]:
LIMIT = 100
radius = 500

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes = df['Latitude'],
                                   longitudes= df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [22]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [23]:
toronto_venues.shape

(2230, 7)

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


## Analyze each neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(2230, 274)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [29]:
toronto_grouped.shape

(100, 274)

### Print the top 5 most common venues for each neighborhood

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2    Skating Rink  0.25
3  Clothing Store  0.25
4  Medical Center  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                             Park   0.5
1                       Playground   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1  Fast Food Restaurant  0.11
2        Sandwich Place  0.11
3   Fried Chicken Joint  0.11
4            Beer Store  0.11


----Alderwood,Long Branch----
            venue  f

4  Shopping Mall  0.17


----Downsview,North Park,Upwood Park----
                        venue  freq
0                      Bakery  0.25
1  Construction & Landscaping  0.25
2                        Park  0.25
3            Basketball Court  0.25
4   Middle Eastern Restaurant  0.00


----East Birchmount Park,Ionview,Kennedy Park----
                venue  freq
0    Department Store  0.25
1          Hobby Shop  0.25
2  Chinese Restaurant  0.25
3         Coffee Shop  0.25
4         Yoga Studio  0.00


----East Toronto----
               venue  freq
0               Park   0.4
1        Pizza Place   0.2
2  Convenience Store   0.2
3        Coffee Shop   0.2
4        Yoga Studio   0.0


----Emery,Humberlea----
                        venue  freq
0  Construction & Landscaping   0.5
1              Baseball Field   0.5
2                 Yoga Studio   0.0
3   Middle Eastern Restaurant   0.0
4         Monument / Landmark   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0   

                  venue  freq
0           Coffee Shop  0.11
1            Restaurant  0.04
2                  Café  0.04
3                 Hotel  0.03
4  Fast Food Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----The Annex,North Midtown,Yorkville----
            venue  freq
0     Coffee Shop  0.13
1  Sandwich Place  0.13
2            Café  0.13
3     Pizza Place  0.09
4  History Museum  0.04


----The Beaches----
                  venue  freq
0     Health Food Store   0.2
1  Other Great Outdoors   0.2
2                   Pub   0.2
3                 Trail   0.2
4           Yoga Studio   0.0


----The Beaches West,India Bazaar----
                venue  freq
0         Pizza Place  0.06
1  Italian Restaurant  0.06
2                 Pub  0.06
3       Movie Theater  0.06
4      Sandwich Place  0.06


----The Danforth Wes

### Sort the venues in descending order

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### display the top 10 venues for each neighborhood

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Hotel,Asian Restaurant,Gym,American Restaurant,Breakfast Spot
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Coffee Shop,Pizza Place,Sandwich Place,Doner Restaurant,Diner
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Dance Studio,Pharmacy,Gym,Comic Shop,Electronics Store


## Cluster Neighborhoods

### Use k-means to cluster the neighborhood into 5 clusters

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hakka Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0.0,Moving Target,Bar,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Pharmacy,Women's Store,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Lounge,Cuban Restaurant,Cupcake Shop


### Visualize the neighborhoods

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#         color=rainbow[cluster-1],
        fill=True,
#         fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Moving Target,Bar,Women's Store,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,Scarborough,0.0,Medical Center,Breakfast Spot,Intersection,Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Doner Restaurant,Discount Store,Dive Bar
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Pharmacy,Women's Store,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,0.0,Hakka Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Lounge,Cuban Restaurant,Cupcake Shop
5,Scarborough,0.0,Playground,Convenience Store,Women's Store,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,0.0,Department Store,Coffee Shop,Hobby Shop,Chinese Restaurant,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,0.0,Bus Line,Park,Intersection,Bakery,Metro Station,Bus Station,Soccer Field,Fast Food Restaurant,Dumpling Restaurant,Drugstore
8,Scarborough,0.0,Motel,American Restaurant,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
10,Scarborough,0.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run


### Cluster 2

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
25,North York,1.0,Fast Food Restaurant,Food & Drink Shop,Park,Grocery Store,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
30,North York,1.0,Park,Airport,Bus Stop,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
40,East York,1.0,Park,Pizza Place,Coffee Shop,Convenience Store,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
50,Downtown Toronto,1.0,Park,Playground,Building,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Empanada Restaurant,Ethiopian Restaurant,Department Store
74,York,1.0,Park,Fast Food Restaurant,Market,Women's Store,Gourmet Shop,Golf Course,Grocery Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant
90,Etobicoke,1.0,Park,River,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store,Doner Restaurant
98,York,1.0,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,College Stadium
100,Etobicoke,1.0,Park,Mobile Phone Shop,Pizza Place,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


### Cluster 3

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,2.0,Cafeteria,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Women's Store,College Stadium


### Cluster 4

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hakka Restaurant


### Cluster 5

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4.0,Home Service,Garden,Music Venue,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
96,North York,4.0,Home Service,Empanada Restaurant,Women's Store,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
